In [10]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 

import import_ipynb
from Environments import Square_Crossroads
from Actor_Critic import Actor_Critic

In [11]:
n_times_steps = 3
seed = 10
dist_cars = 0.5

env = Square_Crossroads(n_times_steps, seed, dist_cars)
env.reset()
print(env.states)
action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

state_tensor = torch.zeros([4,2], dtype=torch.float32)


ac = Actor_Critic(state_tensor, 20, 400)

Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}


In [16]:
hyperparameters = {"hidden_size_actor": 20,
                   "hidden_size_critic": 40,
                   "lr": 3e-4,}

max_episodes = 300
num_steps = 1000
GAMMA = 0.99


def train(env, hyperparameters):
    
    model = Actor_Critic(state_tensor, hyperparameters["hidden_size_actor"], hyperparameters["hidden_size_critic"])
    model_optimizer = optim.Adam(model.parameters(), lr = hyperparameters["lr"])

    all_lengths = []
    average_lengths = []
    all_rewards = []
    entropy_term = 0

    for episode in range(max_episodes):
        print("Training episode: ", episode)
        
        log_probs = []
        values = []
        rewards = []

        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)

        for steps in tqdm(range(num_steps)):
            print(f"Step {steps} of episode {episode}")
            
            value, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
            
            value = torch.sum(value).detach().numpy()
            
#             dist = policy_dist.detach().numpy() 

            action_1 = np.random.beta(policy_dist_alpha[0].detach().numpy(), policy_dist_beta[0].detach().numpy(), size=2)
            action_2 = np.random.beta(policy_dist_alpha[1].detach().numpy(), policy_dist_beta[1].detach().numpy(), size=2)
            action_3 = np.random.beta(policy_dist_alpha[2].detach().numpy(), policy_dist_beta[2].detach().numpy(), size=2)
            action_4 = np.random.beta(policy_dist_alpha[3].detach().numpy(), policy_dist_beta[3].detach().numpy(), size=2)

            action_space = torch.tensor([action_1,
                                         action_2,
                                         action_3,
                                         action_4],
                                         dtype=torch.float32)
            
#             log_prob = torch.log(policy_dist.squeeze(0)[action])            
#             entropy = -np.sum(np.mean(dist) * np.log(dist))
            
            new_state_space, reward, done = env.step(action_space)

            rewards.append(reward)
            values.append(value)
#             log_probs.append(log_prob)
#             entropy_term += entropy
            
            states = [v[0] for v in env.states.values()] 
            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)

            state_space = new_state_space
            
            if done or steps == num_steps-1:
                Qval, policy_dist_alpha, policy_dist_beta = model.forward(state_space)
                Qval = torch.sum(Qval).detach().numpy()
                all_rewards.append(np.sum(rewards))
                all_lengths.append(steps)
                average_lengths.append(np.mean(all_lengths[-10:]))
                if episode % 10 == 0:                    
                    sys.stdout.write("episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards), steps, average_lengths[-1]))
                break
        
        # compute Q values
        Qvals = np.zeros_like(values)
        for t in reversed(range(len(rewards))):
            Qval = rewards[t] + GAMMA * Qval
            Qvals[t] = Qval
        
        print(values)
        #update actor critic
        
        values = [torch.from_numpy(item).float() for item in values]
        Qvals = torch.FloatTensor(Qvals)
        log_probs = torch.stack(log_probs)
        
        advantage = Qvals - values
        actor_loss = (-log_probs * advantage).mean()
        critic_loss = 0.5 * advantage.pow(2).mean()
        ac_loss = actor_loss + critic_loss + 0.001 * entropy_term

        ac_optimizer.zero_grad()
        ac_loss.backward()
        ac_optimizer.step()

        
    
#     # Plot results
#     smoothed_rewards = pd.Series.rolling(pd.Series(all_rewards), 10).mean()
#     smoothed_rewards = [elem for elem in smoothed_rewards]
#     plt.plot(all_rewards)
#     plt.plot(smoothend_rewards)
#     plt.plot()
#     plt.xlabel('Episode')
#     plt.ylabel('Reward')
#     plt.show()

#     plt.plot(all_lengths)
#     plt.plot(average_lengths)
#     plt.xlabel('Episode')
#     plt.ylabel('Episode length')
#     plt.show()

In [17]:
train(env, hyperparameters)

Training episode:  0
tensor([[8.8464, 8.7093],
        [9.3869, 9.2170],
        [9.8643, 9.9314],
        [9.9613, 9.1836]])






  0%|                                                                                                        | 0/1000 [00:00<?, ?it/s]

Step 0 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 1 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 2 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 3 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 4 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 5 of episode 0
A car tried to go outside the boundaries
A car tried to go o





  1%|█                                                                                             | 11/1000 [00:00<00:09, 107.84it/s]

Step 11 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 12 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 13 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 14 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 15 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 16 of episode 0
A car tried to go outside the boundaries
A car tried t

...





 99%|████████████████████████████████████████████████████████████████████████████████████████████▎| 993/1000 [00:09<00:00, 104.20it/s]

Step 993 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 994 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 995 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 996 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 997 of episode 0
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
Step 998 of episode 0
A car tried to go outside the boundaries
A car t

 99%|████████████████████████████████████████████████████████████████████████████████████████████▎| 993/1000 [00:09<00:00, 107.42it/s]


[array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3085399, dtype=float32), array(2.3

RuntimeError: stack expects a non-empty TensorList